In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn
import seaborn as sns
from matplotlib import pyplot as plt




In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed()

In [2]:
dir = "isic-2024-challenge"
train_df = pd.read_csv(f"{dir}/train-metadata.csv")
train_img = f"{dir}/train-image/image"


/var/folders/3_/vryb3cvs6cjffdgbdr66z8dw0000gn/T/ipykernel_1174/1687473423.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{dir}/train-metadata.csv")


In [3]:
train_df.describe()

,target,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,mel_thick_mm,tbp_lv_dnn_lesion_confidence
count,401059.000000,398261.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,...,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,401059.000000,63.000000,4.010590e+05
mean,0.000980,58.012986,3.930827,19.974007,14.919247,28.281706,26.913015,34.786341,30.921279,54.653689,...,1.016459,2.715190,2.238605,0.306823,86.332073,-3.091862,1039.598221,55.823389,0.670952,9.716220e+01
std,0.031288,13.596165,1.743068,3.999489,3.529384,5.278676,4.482994,5.708469,4.829345,5.520849,...,0.734631,1.738165,0.623884,0.125038,52.559511,197.257995,409.819653,87.968245,0.792798,8.995782e+00
min,0.000000,5.000000,1.000000,-2.487115,-9.080269,-0.730989,9.237066,3.054228,11.846520,-1.574164,...,0.000000,0.268160,0.636247,0.052034,0.000000,-624.870728,-1052.134000,-291.890442,0.200000,1.261082e-16
25%,0.000000,50.000000,2.840000,17.330821,12.469740,24.704372,23.848125,31.003148,27.658285,51.566273,...,0.563891,1.456570,1.834745,0.211429,40.000000,-147.022125,746.519673,-8.962647,0.300000,9.966882e+01
50%,0.000000,60.000000,3.370000,19.801910,14.713930,28.171570,26.701704,34.822580,30.804893,55.035632,...,0.902281,2.186693,2.149758,0.282297,90.000000,-5.747253,1172.803000,67.957947,0.400000,9.999459e+01
75%,0.000000,70.000000,4.380000,22.304628,17.137175,31.637429,29.679913,38.430298,33.963868,58.298184,...,1.334523,3.474565,2.531443,0.382022,130.000000,140.474835,1342.131540,126.611567,0.600000,9.999996e+01
max,1.000000,85.000000,28.400000,48.189610,37.021680,54.306900,48.372700,58.765170,54.305290,105.875784,...,11.491140,17.563650,25.534791,0.977055,175.000000,614.471700,1887.766846,319.407000,5.000000,1.000000e+02


In [ ]:
def plot_nan_heatmap(
    df: pd.DataFrame, 
    figsize: tuple = (17, 8), 
    cmap: str = 'magma_r', 
    title: str = 'NaN Values in DataFrame',
    x_tick_rotation=60,
    show_cbar: bool = False, 
    show_yticklabels: bool = False
) -> None:
    """Create a heatmap to visualize NaN values in a DataFrame.

    Args:
        df (pd.DataFrame): 
            The input DataFrame to visualize.
        figsize (tuple[int], optional): 
            Figure size as a tuple of (width, height)
        cmap (str, optional): 
            Colormap to use for the heatmap
        title (str, optional): 
            Title for the heatmap.
        x_tick_rotation (int, optional): 
        Rotation angle for x-axis tick labels.
        show_cbar (bool, optional): 
            Whether to show the color bar.
        show_yticklabels (bool, optional): 
            Whether to show y-axis tick labels.

    Returns:
        None; 
            The function displays the plot using plt.show().
    """
    
    # Setup the figure
    plt.figure(figsize=figsize)
    # Create the heatmap
    sns.heatmap(df.isna(), cbar=show_cbar, yticklabels=show_yticklabels, cmap=cmap)
    
    # Update the title/labels
    plt.title(title, fontweight="bold")
    plt.xlabel('Columns', fontweight="bold")
    plt.ylabel('Rows', fontweight="bold")
    
    
    # Rotate x-axis labels
    plt.xticks(rotation=x_tick_rotation, ha='right')
    
    # Adjust the bottom margin to prevent label cutoff
    plt.tight_layout()
    
    # Render
    plt.show()
     # Print NaN counts per column
    nan_counts = df.isna().sum().sort_values(ascending=False)
    
    # clr_print()
    print("NaN counts per column:")
    print(nan_counts[nan_counts])
    print("Features with 0 NaN values:")
    